In [1]:
import pandas as pd
import os
import polars as pl
import pickle

In [2]:
DATA_ROOT = "./data"
DATASET_PATH = "ebnerd_testset/ebnerd_testset"

In [3]:
# processing articles
articles = pl.read_parquet(os.path.join(DATA_ROOT, DATASET_PATH, "articles.parquet"))
articles = articles.with_columns(
    pl.col("image_ids").map_elements(lambda x: x[0] if len(x) > 0 else None).alias("image_id")
)
articles = articles.with_columns(
    pl.col("topics").map_elements(lambda x: len(x)).alias("topics_count")
)
articles = articles.with_columns(
    pl.col("topics").map_elements(lambda x: x[0].lower() if len(x) > 0 else None).alias("topic")
)
articles = articles.with_columns(
    pl.col("category_str").str.to_lowercase().alias("category")
)
articles = articles[["article_id", "premium", "image_id", "article_type", "topic", "topics_count", "category", "sentiment_score"]]
articles.columns = ["article_id", "is_premium", "image_id", "article_type", "topic", "topics_count", "category", "sentiment_score"]

In [4]:
# processing history
history = pl.read_parquet(os.path.join(DATA_ROOT, DATASET_PATH, "test", "history.parquet"))
history = history.drop("scroll_percentage_fixed")
history = history.explode(["impression_time_fixed", "article_id_fixed", "read_time_fixed"])
history = (
    history
    .sort(['user_id', 'impression_time_fixed'], descending=[False, True])
    .groupby('user_id')
    .agg([
        pl.all().head(20)
    ])
    .explode(pl.all().exclude("user_id"))
)

/scratch-local/dseputis.6680505/ipykernel_2245280/2686497832.py:8: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby('user_id')


In [5]:
# processing behaviors
behaviors = pl.read_parquet(os.path.join(DATA_ROOT, DATASET_PATH, "test", "behaviors.parquet"))

inviews = behaviors.select("article_ids_inview")

behaviors = behaviors.drop('session_id')
behaviors = behaviors.with_row_index(name='session_id')
behaviors = behaviors.rename({col: col.replace("index", "session_id") for col in behaviors.columns})

behaviors = behaviors[["session_id", "impression_id", "user_id"]]
behaviors = behaviors.join(history, on="user_id", how="left")
behaviors.columns = ["session_id", "impression_id", "user_id", "impression_time", "article_id", "read_time"]

behaviors = behaviors.join(articles, on="article_id", how="left")

In [6]:
inviews.write_parquet(os.path.join(DATA_ROOT, DATASET_PATH, "test", "inviews.parquet"))
behaviors.write_parquet(os.path.join(DATA_ROOT, DATASET_PATH, "all.parquet"))

In [7]:
df = pl.read_parquet(os.path.join(DATA_ROOT, DATASET_PATH, "all.parquet"))